Import packages and processed run metrics

In [89]:
def df_to_latex(
    df,
    id_columns,
    file_name,
    remove_columns=None,
    file_path='output'
):

    # remove columns not of interest
    if remove_columns is None:
        remove_columns = list()
    temp_df = df[[name for name in df.columns if name not in remove_columns]]

    # sort by id columns
    temp_df = temp_df.sort_values(by=id_columns)

    # create output file
    with open(os.path.join(file_path, f'{file_name}.txt'), 'w') as f:
        string_df = temp_df.to_string(
            header=True,
            index=False,
            index_names=False
        )

        # remove any leading space
        string_df = re.sub('^[\s]+', '', string_df)
        # remove leading space after newlines
        string_df = re.sub('\n[\s]+', '\n', string_df)
        # replace white space between words with table column skip
        string_df = re.sub('[ \t]+', ' & ', string_df)

        # add latex newline to end of each line
        string_df = string_df.replace('_', ' ')
        string_df = string_df.replace('\n', ' \\\\\n')

        # write to file
        f.writelines(string_df + ' \\\\')

    return None

In [90]:
# packages
import os
import re
import numpy as np
import pandas as pd

# source
helper = __import__('00_helper')

# control
project_names = [
    'femnist--s02', 'sst2', 'pubmed',
    #'cifar--alpha5.0', 'cifar--alpha0.5', 'cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

df = helper.load_data(project_names)


input datasets:
	 (413, 35)
	 (1200, 44)
	 (780, 37)
all runs: (2393, 44)


Filter data to runs of interest
Apply summary function to filtered dataset

In [91]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches

subset_df = subset_df.loc[(subset_df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', subset_df.shape)

subset_df = subset_df.loc[(subset_df.K < 6) | (subset_df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', subset_df.shape)

subset_df = subset_df.loc[((10 * subset_df.beta).astype('Int64') % 2 == 0) | (subset_df.method != 'exact')]
print('\t reduce beta grid for decay:', subset_df.shape)

## column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if (
        not re.search('test', name)
        and not re.search('f1', name)
        and not re.search('loss', name)
    )
]]

def convert_SI(x):
    if pd.isna(x):
        return x

    SI = {
        'K': 10e3,
        'M': 10e6,
        'G': 10e9,
        'T': 10e12
    }

    units = re.sub('[0-9.]', '', x)
    x = float(re.sub('[^0-9.]', '', x))

    if units == '':
        return x
    return x * SI[units]


def to_SI(x, unit='G', decimals=2):
    SI = {
        'K': 10e3, 'M': 10e6,
        'G': 10e9, 'T': 10e12
    }

    assert unit in [name for name, _ in SI.items()]
    return str(round(x / SI[unit], decimals)) + f' {unit}'


#convert_SI('19.0M')

input dataset: (2393, 44)
	 reduce to 3 or fewer local update steps: (1787, 44)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (1707, 44)
	 reduce beta grid for decay: (1610, 44)


In [92]:
subset_df['sys_avg/total_model_size'].unique()

array(['12.53M', '6.27M', '18.8M', nan, '4.1M', '12.3M', '8.2M', '33.45K',
       '66.9K', '100.35K'], dtype=object)

In [93]:
## get best runs for each group
top_k = 3
print('before:', subset_df.shape)
best_runs = subset_df \
    .sort_values(by=validation_metric, ascending=False) \
    .groupby(['dataset', 'method', 'finetune']) \
    .head(top_k)
print('after:', best_runs.shape)

# columns of interest
id_columns = ['dataset', 'method']
metrics = [name for name in best_runs.columns if re.search('^sys', name)]
print('metrics:', metrics)
object_metrics= [
    name for name, types in zip(best_runs.columns, best_runs.dtypes)
    if re.search('^sys', name)
    and types == 'object'
]

# subset to non-fine-tuned models
float_df = best_runs.loc[best_runs.finetune == 0]
print('without FT:', float_df.shape)
float_df = float_df[id_columns + metrics]
float_df[object_metrics] = float_df[object_metrics].applymap(convert_SI).apply(pd.to_numeric)
float_df['sys_avg/total_bytes'] = float_df['sys_avg/total_download_bytes'] \
                          + float_df['sys_avg/total_upload_bytes']


before: (1610, 20)
after: (102, 20)
metrics: ['sys_avg/global_convergence_time_minutes', 'sys_avg/total_model_size', 'sys_avg/fl_end_time_minutes', 'sys_avg/total_flops', 'sys_avg/local_convergence_time_minutes', 'sys_avg/local_convergence_round', 'sys_avg/global_convergence_round', 'sys_avg/total_upload_bytes', 'sys_avg/total_download_bytes']
without FT: (51, 20)


In [94]:
sub_metrics = [
    'sys_avg/global_convergence_round',
    'sys_avg/global_convergence_time_minutes',
    'sys_avg/total_bytes',
    'sys_avg/total_flops'
]
avg_runs = float_df.groupby(id_columns)[sub_metrics].mean()

avg_runs['sys_avg/global_convergence_round'] = avg_runs['sys_avg/global_convergence_round'].apply(round, ndigits=2)
avg_runs['sys_avg/global_convergence_time_minutes'] = avg_runs['sys_avg/global_convergence_time_minutes'].apply(round, ndigits=2)
avg_runs['sys_avg/total_bytes'] = avg_runs['sys_avg/total_bytes'].apply(to_SI, unit='M')
avg_runs['sys_avg/total_flops'] = avg_runs['sys_avg/total_flops'].apply(to_SI, unit='G')
avg_runs = avg_runs.add_suffix('_mean').reset_index()

id_columns = ['method', 'dataset']
avg_runs.columns = [re.sub('sys_avg/', '', name) for name in avg_runs.columns]
df_to_latex(avg_runs, id_columns, 'method_costs--ft_no')